# Example of standard (noiseless) GP regression 
- Create a GP model quickly using ```rapid_models/gp_models```
- Fit GP model to a (randomly generated) dataset
- Generate interactive diagnostics plots using ```rapid_models/gp_diagnostics_plotly```

In [41]:
import os, sys
import numpy as np
from datetime import datetime

import plotly.graph_objects as go

# The rapid_models GP models are created using gpytorch
import torch
import gpytorch
print('torch', torch.__version__)
print('gpytorch', gpytorch.__version__)

# Import a template and some helper functions to quickly create a GP model 
# *** Alternatively: copy the boilerplate code from the ExactGPModel and modify ***
sys.path.append('../..') # Add the path of rapid_models (when cloned from git and not installed using pip)
from rapid_models.gp_models.templates import ExactGPModel 
from rapid_models.gp_models.utils import gpytorch_kernel_Matern, gpytorch_mean_constant, gpytorch_likelihood_gaussian

torch 1.9.0+cpu
gpytorch 1.5.0


### Generate some random training data 

In [2]:
## Set some GLOBAL test variables
N_DIM = 3         # input dimension
N_TRAIN = 100     # number of training samples
N_TEST = 100      # number of test samples

## Will generate data using a zero-mean Matern 5/2 GP with these parameters
KER_SCALE_TRUE = 1.0
KER_LENGTHSCALE_TRUE = torch.ones(N_DIM)*0.5

In [3]:
## Generate some dummy training/test data from a GP
torch.manual_seed(0)

ker = gpytorch_kernel_Matern(KER_SCALE_TRUE, KER_LENGTHSCALE_TRUE)
X = torch.rand(size=(N_TRAIN + N_TEST, N_DIM))
normal_rv = gpytorch.distributions.MultivariateNormal(mean = torch.zeros(N_TRAIN + N_TEST), covariance_matrix = ker(X))
Y = normal_rv.sample()

X_train, X_test = X[0:N_TRAIN], X[N_TRAIN:]
Y_train, Y_test = Y[0:N_TRAIN], Y[N_TRAIN:]

### Create GP model with randomly initialized hyperparameters

In [37]:
## The optimal GP 
model = ExactGPModel(
                     X_train, Y_train,                                                                    # Training data
                     gpytorch_mean_constant(0.0, fixed = True),                                           # Mean function
                     gpytorch_kernel_Matern(var = torch.rand(1), ls = torch.rand(N_DIM)),                 # Kernel
                     gpytorch_likelihood_gaussian(variance = 1e-6, fixed = True),                         # Likelihood
                     '', '') # Name and path for save/load

model.print_parameters()

Constant mean                                      0.0
Likelihood noise variance                          9.999999974752427e-07
Kernel lengthscale                                 [0.34716582 0.77170414 0.35698533]
Kernel outputscale (variace)                       0.9951531887054443


### Optimize GP hyperparameters by maximizing the marginal likelihood
Here we force gpytorch to use the common Cholesky-based implementation by setting ```gpytorch.settings.max_cholesky_size``` larger than the number of training data ```N_TRAIN```. If this is set below ```N_TRAIN```, gpytorch will use the BBMM (BlackBox Matrix-Matrix) implementation. 

In [38]:
def step(model, loss_function, optimizer):
    """
    Return current loss and perform one optimization step
    """
    # zero the gradients
    optimizer.zero_grad()

    # compute output from model
    output = model(model.train_inputs[0])

    # calc loss and backprop gradients
    loss = -loss_function(output, model.train_targets)
    loss.backward()
    optimizer.step()

    return loss.item()  

In [39]:
# Training mode
model.train_mode()
model._clear_cache()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 

# Likelihood function
mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)

# Number of training epochs
num_epochs = 100

# For plotting
losses, ls, scale = [], [], []

t0=datetime.now()
with gpytorch.settings.max_cholesky_size(N_TRAIN*2), gpytorch.settings.min_preconditioning_size(N_TRAIN*2), gpytorch.settings.fast_computations(covar_root_decomposition=False, log_prob=False, solves=False):
    for i in range(num_epochs):
        losses.append(step(model, mll, optimizer))
        ls.append(model.covar_module.base_kernel.lengthscale.detach().numpy()[0])
        scale.append(model.covar_module.outputscale.detach().item())
print('Done. {} epochs, time = {}'.format(num_epochs, datetime.now()-t0))   
ls = np.array(ls)

Done. 100 epochs, time = 0:00:00.328084


In [55]:
# Plot the loss after each iteration
fig = go.Figure(data=go.Scatter(y=losses, mode='lines'))
fig.update_xaxes(title='Iteration')
fig.update_yaxes(title='Loss')

fig.show()

print('Updated model parameters: \n')
model.print_parameters()

Updated model parameters: 

Constant mean                                      0.0
Likelihood noise variance                          9.999999974752427e-07
Kernel lengthscale                                 [0.41261277 0.58322626 0.5197337 ]
Kernel outputscale (variace)                       0.8878334164619446


### Evaluate GP predictions